In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score
from plot_helpers import plot_interpolated
import os
import main
import keras
import masked_metrics
import models

In [ ]:
train_ds, test_ds, val_ds, Xs, Xc, Y, ds = main.construct_dataset(100)

In [12]:
Y_inter_s_c     = np.load(os.path.normpath('runs/2025-12-08_20-28-11/y_inter.npz'))['arr_0']
Y_pred_s_c      = np.load(os.path.normpath('runs/2025-12-08_20-28-11/y_pred.npz'))['arr_0']
Y_inter_s_nc    = np.load(os.path.normpath('runs/2025-12-08_20-39-41/y_inter.npz'))['arr_0']
Y_pred_s_nc     = np.load(os.path.normpath('runs/2025-12-08_20-39-41/y_pred.npz'))['arr_0']
Y_inter_ns_c    = np.load(os.path.normpath('runs/2025-12-08_20-07-00/y_inter.npz'))['arr_0']
Y_pred_ns_c     = np.load(os.path.normpath('runs/2025-12-08_20-07-00/y_pred.npz'))['arr_0']
Y_inter_ns_nc   = np.load(os.path.normpath('runs/2025-12-08_20-54-44/y_inter.npz'))['arr_0']
Y_pred_ns_nc    = np.load(os.path.normpath('runs/2025-12-08_20-54-44/y_pred.npz'))['arr_0']

arg_list = [
    (Y_pred_s_c, True, True), 
    (Y_pred_s_nc, True, False), 
    (Y_pred_ns_c, False, True),
    (Y_pred_ns_nc, False, False)
]

In [32]:
def r2(Y_true:np.ndarray, Y_pred:np.ndarray, seq2seq:bool, context:bool, display:bool=True): 
    Y_true_sliced = Y_true[:,-1,:]
    Y_pred_sliced = Y_pred
    if seq2seq: 
        Y_pred_sliced = Y_pred[:,-1,:] 
    Y_cat = np.concat([Y_pred_sliced, Y_true_sliced], -1) 
    Y_cat = Y_cat[~np.isnan(Y_cat).any(axis=-1)] 
    Y_cat = np.stack([Y_cat[:,:5], Y_cat[:,5:]], axis=0)
    score = r2_score(Y_cat[1], Y_cat[0], multioutput='raw_values') 
    if display:
        print(
            f"seq2seq={seq2seq}, context={context}:\t{score}"
        )
    return score

Y_np = Y.numpy()
list(map(lambda T: r2(Y_np, *T), arg_list))

seq2seq=True, context=True:	[-0.15669739  0.6986846   0.67707956  0.47706258  0.40751237]
seq2seq=True, context=False:	[-0.5997884   0.7324649  -2.3261232   0.20671624 -2.2759335 ]
seq2seq=False, context=True:	[-1.9417727   0.7128873   0.57206625 -0.48760402 -2.839726  ]
seq2seq=False, context=False:	[-0.61462975  0.7392696   0.5153121   0.68986404  0.25727916]


[array([-0.15669739,  0.6986846 ,  0.67707956,  0.47706258,  0.40751237],
       dtype=float32),
 array([-0.5997884 ,  0.7324649 , -2.3261232 ,  0.20671624, -2.2759335 ],
       dtype=float32),
 array([-1.9417727 ,  0.7128873 ,  0.57206625, -0.48760402, -2.839726  ],
       dtype=float32),
 array([-0.61462975,  0.7392696 ,  0.5153121 ,  0.68986404,  0.25727916],
       dtype=float32)]